# OPENAI

In [ ]:
import openai
openai.api_key = ""

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

# Prompt generation

In [2]:
import random

In [3]:
ways = {}
angles = [list(range(-65,-45)),list(range(-45,-25)),list(range(-25,-10)),list(range(-10,10)),list(range(10,25)),list(range(25,45)),list(range(45,65))]
directions = ["full left","half left","slightly left","forward","slightly right","half right","full right"]
for l,w in zip(angles,directions):
    ways.update({k:w for k in l})

no_yolo_example_output = [(1, 25, 0), (2, -30, 1), (7, 40, 2)]
yes_yolo_example_output = [(1, 25, 0, "Tree"), (2, -30, 1, "Car"), (7, 40, 2, "Table")]

In [4]:
# generates prompt by merging infos from models outputs
def generate_prompt(model_out):
    if len(model_out) == 0:
        return "no obstacle detected, go forward"

    state_str = ""
    closed_directions = set()
    
    if len(model_out[0]) == 3: # yolo didnt detect anything
        for i in model_out:
            closed_directions.add(ways[i[1]])
            state_str = state_str + ("obstacle at " + str(i[0]) + " meters ahead at " + str(ways[i[1]])) + "\n"
    else: # yolo detected something
        for i in model_out:
            closed_directions.add(ways[i[1]])
            state_str = state_str + (i[3] + " at " + str(i[0]) + " meters ahead at " + str(ways[i[1]])) + "\n"

    if "forward" not in closed_directions:
        recommended_direction = "forward"
    else:
        open_directions = [x for x in directions if x not in closed_directions]
        recommended_direction = random.choice(open_directions)

    with open("../LLM_prompts/ready_prompt", 'r') as file:
        ready_prompy = file.read()
    state_str = ready_prompy + state_str + "user should go " + recommended_direction
    state_str = state_str + "\n\nNow write a short sentence to guide the user."

    return state_str

In [5]:
print(generate_prompt(no_yolo_example_output))

There is a system to help blind people, this system captures the obstacles ahead the user via camera and calculates how far and how many angles left or right are this obstacles from user. You should guide the user. Now I will provide the outputs of this system and you will give clear instructions based on where are the obstacles and guide the user with short sentences, don’t output anything except guidance sentences. Here is the captured information of the scene(output only short sentences to guide user):

obstacle at 1 meters ahead at half right
obstacle at 2 meters ahead at half left
obstacle at 7 meters ahead at half right
user should go forward

Now write a short sentence to guide the user.


In [6]:
print(generate_prompt(yes_yolo_example_output))

There is a system to help blind people, this system captures the obstacles ahead the user via camera and calculates how far and how many angles left or right are this obstacles from user. You should guide the user. Now I will provide the outputs of this system and you will give clear instructions based on where are the obstacles and guide the user with short sentences, don’t output anything except guidance sentences. Here is the captured information of the scene(output only short sentences to guide user):

Tree at 1 meters ahead at half right
Car at 2 meters ahead at half left
Table at 7 meters ahead at half right
user should go forward

Now write a short sentence to guide the user.
